# Importando Bibliotecas

In [101]:
import pandas as pd
import geopandas as gpd
import glob
import folium
import os
from folium.plugins import MarkerCluster
from shapely.geometry import Polygon, Point, LineString, MultiPolygon

# Importando dados

* Carregando todos os arquivos de uma só vez e concatenando em um df principal

In [102]:
path = 'data/'                     
todos_arquivos = glob.glob(os.path.join(path, "*.csv"))    

dfs = (pd.read_csv(arquivo, usecols=['DATAOCORRENCIA', 'PERIDOOCORRENCIA','LOGRADOURO', 'NUMERO', 'CIDADE', 
                                     'UF', 'LATITUDE', 'LONGITUDE', 'PLACA_VEICULO', 'DESCR_COR_VEICULO', 'DESCR_MARCA_VEICULO', 
                                     'ANO_MODELO', 'DESCR_TIPO_VEICULO'], 
                   sep=";", decimal=',', encoding='latin1') for arquivo in todos_arquivos)

df = pd.concat(dfs, ignore_index=True)

## Verificando Base de Dados

In [103]:
#Visualizando as 5 primeiras linhas 
df.head()

,DATAOCORRENCIA,PERIDOOCORRENCIA,LOGRADOURO,NUMERO,CIDADE,UF,LATITUDE,LONGITUDE,PLACA_VEICULO,DESCR_COR_VEICULO,DESCR_MARCA_VEICULO,ANO_MODELO,DESCR_TIPO_VEICULO
0,31/12/2020,A NOITE,RUA PROCÓPIO BARBOSA DE MIRANDA,128,ITAQUAQUECETUBA,SP,-23.464688,-46.316572,DRW2B37,Verde,YAMAHA/YBR 125K,NaN,MOTOCICLO
1,31/12/2020,A NOITE,RUA DOUTOR JOSE PEREIRA GOMES,1627,SÃO PAULO,SP,-23.496574,-46.372711,EZP9F19,Cinza,FORD/KA SE 1.0 HA C,NaN,AUTOMOVEL
2,31/12/2020,A NOITE,AVENIDA GABRIELA MISTRAL,95,SÃO PAULO,SP,-23.516960,-46.549701,DRL4572,Branco,RENAULT/KWID ZEN 10MT,2020.0,AUTOMOVEL
3,31/12/2020,A NOITE,RODOVIA SP 324,520,CAMPINAS,SP,-23.024741,-47.125820,QMU9905,Prata,RENAULT/SANDERO,NaN,AUTOMOVEL
4,31/12/2020,A NOITE,RUA DOCE VITORIA,5,SÃO PAULO,SP,-23.567778,-46.418146,FAY7731,Prata,CHEVROLET/ONIX 1.4MT LT,2015.0,AUTOMOVEL


In [104]:
# Verificando tamanho da base de dados
print('{} Linhas e {} colunas'.format(df.shape[0], df.shape[1]))

37702 Linhas e 13 colunas


In [105]:
df.dtypes

DATAOCORRENCIA          object
PERIDOOCORRENCIA        object
LOGRADOURO              object
NUMERO                   int64
CIDADE                  object
UF                      object
LATITUDE               float64
LONGITUDE              float64
PLACA_VEICULO           object
DESCR_COR_VEICULO       object
DESCR_MARCA_VEICULO     object
ANO_MODELO             float64
DESCR_TIPO_VEICULO      object
dtype: object

* Alterando os tipos de colunas

In [106]:
df[['NUMERO', 'ANO_MODELO']] = df[['NUMERO', 'ANO_MODELO']].astype('object')

In [107]:
df.dtypes

DATAOCORRENCIA          object
PERIDOOCORRENCIA        object
LOGRADOURO              object
NUMERO                  object
CIDADE                  object
UF                      object
LATITUDE               float64
LONGITUDE              float64
PLACA_VEICULO           object
DESCR_COR_VEICULO       object
DESCR_MARCA_VEICULO     object
ANO_MODELO              object
DESCR_TIPO_VEICULO      object
dtype: object

* Verificando se há dados faltantes

In [108]:
print('Quantidade de dados nulos:', df.isna().sum().sum())

Quantidade de dados nulos: 48842


In [109]:
 df.isna().sum()

DATAOCORRENCIA             0
PERIDOOCORRENCIA           0
LOGRADOURO                 0
NUMERO                     0
CIDADE                     0
UF                         0
LATITUDE                   0
LONGITUDE                  0
PLACA_VEICULO          10852
DESCR_COR_VEICULO       9708
DESCR_MARCA_VEICULO    10665
ANO_MODELO              7909
DESCR_TIPO_VEICULO      9708
dtype: int64

### Decisão sobre os dados Ausentes.

* Como os dados faltantes se tratam de uma informção sobre o veiculo roubado, não faz sentido remover eses dados.
* Diante disso irei apenas atribuir 'Não informando' a esses dados faltantes.

In [110]:
df.fillna( 'NÃO INFORMADO', inplace=True)
print('Quantidade de dados nulos:', df.isna().sum().sum())

Quantidade de dados nulos: 0


# Plotando Mapas

* Carregando dados de geometria que serão usados para plotagem de mapas

In [111]:
#importando arquivo com a geometria dos estado
uf_sp = gpd.read_file('mapas/ESTADOS/35UFE250GC_SIR.shp')

## Transformando DataFrame em GeoDataDrame

In [112]:
# filtrando apenas a localização
lat_long = list(zip(df.LATITUDE, df.LONGITUDE))
lat_long

[(-23.464688, -46.3165719),
 (-23.49657433, -46.37271051),
 (-23.51696043, -46.54970075),
 (-23.02474138, -47.12582018),
 (-23.56777787, -46.41814619),
 (-23.52918892, -46.47812736),
 (-23.66183562, -46.68918967),
 (-23.5007773, -46.4068532),
 (-22.95027249, -47.11010612),
 (-22.95027249, -47.11010612),
 (-23.55058685, -46.96160106),
 (-23.7193468, -46.55188598),
 (-23.7193468, -46.55188598),
 (-22.94248343, -47.08920348),
 (-23.47155653, -46.30783202),
 (-23.47155653, -46.30783202),
 (-23.47155653, -46.30783202),
 (-23.47155653, -46.30783202),
 (-23.62351565, -46.54033309),
 (-23.62484027, -47.04726161),
 (-23.62484027, -47.04726161),
 (-23.62484027, -47.04726161),
 (-23.62484027, -47.04726161),
 (-23.82916501, -46.72206729),
 (-23.61877235, -46.76573697),
 (-22.83921673, -47.15588486),
 (-23.43550906, -46.5752803),
 (-23.43550906, -46.5752803),
 (-23.43550906, -46.5752803),
 (-23.43550906, -46.5752803),
 (-23.43550906, -46.5752803),
 (-23.43550906, -46.5752803),
 (-22.91273734, -47.0

In [113]:
#transformando em pontos geometricos
geometry = [Point(x) for x in lat_long]
geometry

 ...]

In [114]:
#configurando crs
crs = {'proj': 'latlong', 'ellps': 'WGS84', 'datum': 'WGS84', 'no_defs': True}
# atribuindo dados geometricos a uma nova coluna do df e transformando em um GEODATAFRAME
geo_dados = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
geo_dados.head()

,DATAOCORRENCIA,PERIDOOCORRENCIA,LOGRADOURO,NUMERO,CIDADE,UF,LATITUDE,LONGITUDE,PLACA_VEICULO,DESCR_COR_VEICULO,DESCR_MARCA_VEICULO,ANO_MODELO,DESCR_TIPO_VEICULO,geometry
0,31/12/2020,A NOITE,RUA PROCÓPIO BARBOSA DE MIRANDA,128,ITAQUAQUECETUBA,SP,-23.464688,-46.316572,DRW2B37,Verde,YAMAHA/YBR 125K,NÃO INFORMADO,MOTOCICLO,POINT (-23.46469 -46.31657)
1,31/12/2020,A NOITE,RUA DOUTOR JOSE PEREIRA GOMES,1627,SÃO PAULO,SP,-23.496574,-46.372711,EZP9F19,Cinza,FORD/KA SE 1.0 HA C,NÃO INFORMADO,AUTOMOVEL,POINT (-23.49657 -46.37271)
2,31/12/2020,A NOITE,AVENIDA GABRIELA MISTRAL,95,SÃO PAULO,SP,-23.516960,-46.549701,DRL4572,Branco,RENAULT/KWID ZEN 10MT,2020.0,AUTOMOVEL,POINT (-23.51696 -46.54970)
3,31/12/2020,A NOITE,RODOVIA SP 324,520,CAMPINAS,SP,-23.024741,-47.125820,QMU9905,Prata,RENAULT/SANDERO,NÃO INFORMADO,AUTOMOVEL,POINT (-23.02474 -47.12582)
4,31/12/2020,A NOITE,RUA DOCE VITORIA,5,SÃO PAULO,SP,-23.567778,-46.418146,FAY7731,Prata,CHEVROLET/ONIX 1.4MT LT,2015.0,AUTOMOVEL,POINT (-23.56778 -46.41815)


In [115]:
# Definindo localização inicial do mapa
y = uf_sp.centroid.y.iloc[0]
x = uf_sp.centroid.x.iloc[0]

C:\WINDOWS\TEMP/ipykernel_2388/2687542583.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  y = uf_sp.centroid.y.iloc[0]
C:\WINDOWS\TEMP/ipykernel_2388/2687542583.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  x = uf_sp.centroid.x.iloc[0]


In [116]:
#setando configurações iniciais do mapa
mapa = folium.Map(location=[y,x], 
                  zoom_start=7, 
                  tiles='OpenStreetMap', alpha=0)

#adicionando contorno do estado de SP
mapa.choropleth(uf_sp[uf_sp['NM_ESTADO'] == "SÃO PAULO"],
                name='São Paulo',
                line_color='black',
                line_weight=1,
                fill_opacity=0)

#atibuindo nome do marcador
cluster = MarkerCluster(name='Carros Roubados')

#atribuindo os dados de roubos, icone e outras configurações do mapa
for item in geo_dados.itertuples():
    cluster.add_child(
        folium.Marker(location=[item.LATITUDE, item.LONGITUDE],
                      popup="TIPO: "+ str(item.DESCR_TIPO_VEICULO)+"<p>MODELO: "+
                      str(item.DESCR_MARCA_VEICULO)+"<p>PLACA: "+str(item.PLACA_VEICULO)+"<p>DATA OCORRÊNCIA: "+str(item.DATAOCORRENCIA),
                      icon= folium.Icon(color='red', 
                                        prefix='fa', 
                                        icon='fas fa-car'), tooltip=item.CIDADE ))

    

#adicionando cluster
mapa.add_child(cluster)
#adicionando controle dos mapas
folium.LayerControl().add_to(mapa)
#salvar o mapa
mapa.save('Roubo de carros.html')

c:\users\raila\documents\github\roubos de carro em sp\virtual\lib\site-packages\folium\folium.py:409: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(
